In [ ]:
%load_ext autoreload
%autoreload 2
from src.inserter import DB
from pathlib import Path
import re
import pandas as pd
import datetime as dt 
import src.queries as queries
import requests
def remove_numeric_prefix_from_columns(df: pd.DataFrame) -> pd.DataFrame:
    """
    Removes patterns like '1. ', '2. ', etc. from DataFrame column names.
    """
    df = df.copy()
    df.columns = [
        re.sub(r"^[0-9]\.\s*", "", col)
        for col in df.columns
    ]
    return df
def add_creation_date(df: pd.DataFrame, tz: dt.timezone = dt.timezone.utc) -> pd.DataFrame:
    df = df.copy()
    df["creation_date"] = dt.datetime.now(tz=tz)
    return df
def apply_fetch_one_with_query(db:DB,row,query):
    if row is pd.Series:
        row = row.to_dict()
    result = db.fetch_one(query,row)
    return result
PROJECT_ROOT = Path.cwd().parent  # adjust if needed
CONFIG_PATH = PROJECT_ROOT /"Atlas"/ "src" / "dbconfig"
db = DB()
db = DB().connect(config_file=str(CONFIG_PATH))
# replace the "demo" apikey below with your own key from https://www.alphavantage.co/support/#api-key
# url = 'https://www.alphavantage.co/query?function=TIME_SERIES_DAILY_ADJUSTED&symbol=SP5H.PAR&apikey=MJV3MROB7WEJ1QNZ'
# r = requests.get(url)
# data = r.json()

# print(data)

In [ ]:
a = apply_fetch_one_with_query(db,{"name":"Equity"},queries.FETCH_ASSET_TYPE_BY_ASSET_TYPE_NAME)

Symbolsearch

Insert asset types based on symbolsearch endpoint

In [ ]:
import pandas as pd
url = 'https://www.alphavantage.co/query?function=SYMBOL_SEARCH&keywords=sp&apikey=MJV3MROB7WEJ1QNZ'
# url = 'https://www.alphavantage.co/query?function=NATURAL_GAS&interval=monthly&apikey=demo'
r = requests.get(url)
data = r.json()
# data["bestMatches"]
data_df = pd.DataFrame(data = data["bestMatches"])
types_df = data_df["3. type"].drop_duplicates()
types_df = types_df.to_frame()
types_df.rename(columns={"3. type":"name"},inplace=True)
# db.copy_df(df = types_df,table="asset_type")

Insert assets

In [ ]:
r = requests.get(url)
data = r.json()
# data["bestMatches"]
data_df = pd.DataFrame(data = data["bestMatches"])
isin_map = {
    "SP5.FRK": "LU0496786574",
    "SP4P.FRK": "LU1135865084",
    "SP5C.LON": "LU0496786574",
    "SP5G.LON": "LU1681048804",
    "SP20.AMS": "LU0496786574",
    "SP2D.DEX": "LU1135865084",
    "SP2Q.DEX": "LU0496786574",
    "SP3C.DEX": "LU1781541179",
    "SP5C.PAR": "LU0496786574",
    "SP5H.PAR": "LU1681048804",
}
data_df["isin"] = data_df["1. symbol"].map(isin_map)
data_df = data_df[["isin","1. symbol","3. type","2. name",]]
# data_df = data_df.rename(columns={"1. symbol":"symbol","3. type":"type","2. name":"name",})
data_df = remove_numeric_prefix_from_columns(data_df)
data_df = add_creation_date(data_df)
data_df["asset_type_id"] = data_df["type"].apply(lambda x:apply_fetch_one_with_query(db,{"name":x},queries.FETCH_ASSET_TYPE_BY_ASSET_TYPE_NAME))[0][0]
data_df = data_df.drop(columns=["type"]).drop_duplicates(subset=["isin"])
# db.copy_df(df=data_df,table="asset")



CURRENCY

In [ ]:
currency_dict = {"USD","EUR","GBP","JPY","CHF","AUD","CAD","NZD","CNY","SEK","NOK","DKK"}
currency_df = pd.DataFrame(list(currency_dict),columns=["name"])
db.copy_df(df=currency_df,table="currency")

In [10]:
data_df

,isin,symbol,asset_type_id,name,creation_date,currency_id
0,LU0496786574,SP5.FRK,4,ASTROTECH CORP.,2025-12-28 12:11:05.081924,12
1,LU0959211243,SP5H.PAR,1,SP500,2025-12-24 18:30:18.737258,12
2,LU1135865084,SP4P.FRK,4,Achieve Life Sciences Inc,2025-12-28 12:11:05.081924,12
3,LU1681048804,SP5G.LON,4,Lyxor S&P 500 UCITS ETF - Daily Hedged to GBP ...,2025-12-28 12:11:05.081924,12
4,LU1781541179,SP3C.DEX,4,iShares S&P 500 3% Capped UCITS ETF USD Acc,2025-12-28 12:11:05.081924,12


ASSET DATA INSERT BASED ON SYMBOL

In [18]:
data_df = db.fetch_df(queries.FETCH_ASSET_BY_ISIN_LIST,{"isin":["LU1781541179","LU1681048804"]})
for index,row in data_df.iterrows():
    symbol = row["symbol"]
    url = f'https://www.alphavantage.co/query?function=TIME_SERIES_DAILY&symbol={symbol}&apikey=MJV3MROB7WEJ1QNZ'
    r = requests.get(url)
    data = r.json()
    sp_data_df = pd.DataFrame(data=data["Time Series (Daily)"])
    sp_data_df = sp_data_df.T
    sp_data_df = sp_data_df.rename(columns={"1. open":"open","2. high":"high","3. low":"low","4. close":"mid_close","5. volume":"volume"})

    isin = apply_fetch_one_with_query(db,{"symbol":row["symbol"]},queries.FETCH_ASSET_ISIN_BY_ASSET_SYMBOL)
    sp_data_df["asset_isin"] = isin[0]
    sp_data_df["date"] = sp_data_df.index
    db.copy_df(df=sp_data_df,table="asset_data")

# print(data)

In [ ]:
data["Meta Data"]

In [ ]:
from src.inserter import DB
from pathlib import Path

PROJECT_ROOT = Path.cwd().parent  # adjust if needed
CONFIG_PATH = PROJECT_ROOT /"Atlas"/ "src" / "dbconfig"
db = DB()
db = DB().connect(config_file=str(CONFIG_PATH))

Asset type and Asset inserts

In [ ]:
import pandas as pd
import datetime as dt
dict = {"name":"Mutual Fund"}
df = pd.DataFrame(data=dict,index=[0])

# db.copy_df(df=df,table="asset_type")
sp_dict = {"isin":"LU0959211243",'symbol': 'SP5H.PAR','asset_type_id':1,"name":"SP500","creation_date":dt.datetime.now(tz=dt.UTC)}
sp_df = pd.DataFrame(data=sp_dict,index=[0])
# db.copy_df(df=sp_df,table="asset")
# "LU0959211243"

In [ ]:
dict = {"name":"Mutual Fund"}
df = pd.DataFrame(data=dict,index=[0])

db.copy_df(df=df,table="asset_type")
sp_dict = {"isin":"LU0959211243",'symbol': 'SP5H.PAR','asset_type_id':1,"name":"SP500","creation_date":dt.datetime.now(tz=dt.UTC)}
sp_df = pd.DataFrame(data=sp_dict,index=[0])
# db.copy_df(df=sp_df,table="asset")
# "LU0959211243"

ISIN query from asset symbol

In [ ]:
isin_query = """SELECT isin FROM public.asset WHERE symbol ='SP5H.PAR' limit 1"""
asset_isin = db.fetch(query=isin_query)
isin = asset_isin[0][0]

Asset data insertion

In [ ]:
data["Time Series (Daily)"]

sp_data_df = pd.DataFrame(data=data["Time Series (Daily)"])
sp_data_df = sp_data_df.T
sp_data_df = sp_data_df.rename(columns={"1. open":"open","2. high":"high","3. low":"low","4. close":"mid_close","5. volume":"volume"})


sp_data_df["asset_isin"] = isin
sp_data_df["date"] = sp_data_df.index
sp_data_df
db.copy_df(df=sp_data_df,table="asset_data")

Asset data fetch

In [ ]:
asset_data_query = """
SELECT *
FROM public.asset_data
WHERE asset_isin = %s
"""

asset_data = db.fetch_df(asset_data_query, params=(isin,))

In [ ]:
import plotly.graph_objects as go
import pandas as pd

# Ensure correct types & order
asset_data["date"] = pd.to_datetime(asset_data["date"])
asset_data_df = asset_data.sort_values("date")

fig = go.Figure(
    data=[
        go.Candlestick(
            x=asset_data_df["date"],
            open=asset_data_df["open"],
            high=asset_data_df["high"],
            low=asset_data_df["low"],
            close=asset_data_df["mid_close"],
            name="Price",
        )
    ]
)

fig.update_layout(
    title="Asset Price (Daily OHLC)",
    xaxis_title="Date",
    yaxis_title="Price",
    xaxis_rangeslider_visible=False,
    template="plotly_white",
)

fig.show()


Profile creation

In [ ]:
import datetime as dt
profile_dict={"name":"Yulian","creation_date":dt.datetime.now(tz=dt.UTC)}
profile_df = pd.DataFrame(data=profile_dict,index=[0])
# db.copy_df(df=profile_df,table="profile")

Transaction insert

In [ ]:
asset_data

In [ ]:
date = asset_data.iloc[40]["date"]
isin = asset_data.iloc[40]["asset_isin"]
price = asset_data.iloc[40]["mid_close"]
transaction_dict = {"date":date,"asset_isin":isin,"user_id":1,"quantity":100,"price":price}
transaction_df = pd.DataFrame(data = transaction_dict,index=[0])
db.copy_df(df = transaction_df,table = "asset_transaction" )